In [1]:
%load_ext autoreload
%autoreload 2

import reflexgenerator.sources
from reflexgenerator.sources import (
    Register, Mask, PinMap, Metadata, Collection)
from reflexgenerator.io import load

from pathlib import Path
import pandas as pd
from reflexgenerator.generator.xref import UidReference


In [2]:
schema = r"schema\device.json"
fw_target = r"schema\example.yml"


In [3]:
jsondata = load(fw_target)


In [4]:
metadata = Metadata(
    **{"device": jsondata["device"],
    "whoAmI": jsondata["whoAmI"],
    "architecture": jsondata["architecture"],
    "firmwareVersion": jsondata["firmwareVersion"],
    "hardwareTargets": jsondata["hardwareTargets"]})
print(metadata)

Metadata(device='ExampleDevice', whoAmI=0, firmwareVersion='0.1', hardwareTargets='0.1', architecture='atmega', uid=ref-Device-Metadata-ExampleDevice_0)


In [5]:
# build masks
if "bitMasks" in jsondata:
    bitMasks = Collection([Mask.from_json(mask) for mask in jsondata["bitMasks"].items() if mask is not None])
    bitMasks_df = pd.DataFrame([mask.to_dict() for mask in bitMasks])
    bitMasks_df.head(5)
else:
    bitMasks = None
    bitMasks_df = None

if "groupMask" in jsondata:
    groupMasks = Collection([Mask.from_json(mask) for mask in jsondata["groupMask"].items() if mask is not None])
    groupMasks_df = pd.DataFrame([mask.to_dict() for mask in groupMasks])
    groupMasks_df.head(5)
else:
    groupMasks = None
    groupMasks_df = None

In [6]:
if "registers" in jsondata:
    regs = Collection([Register.from_json(reg) for reg in jsondata["registers"].items() if reg is not None])
    register_df = pd.DataFrame([reg.to_dict() for reg in regs])
    register_df.head(5)
else:
    regs = None
    register_df = None

In [7]:
if "ios" in jsondata:
    pinMapping = Collection([PinMap.from_json(pinmap) for pinmap in jsondata["ios"].items() if pinmap is not None])
    pinMapping_df = pd.DataFrame([pinmap.to_dict() for pinmap in pinMapping])
    pinMapping_df.head(5)
else:
    pinMapping = None
    pinMapping_df = None

In [18]:
register_df_copy = register_df.copy()
for i in register_df_copy.index:
    if register_df_copy.at[i, 'maskType']:
        if isinstance(register_df_copy.at[i, 'maskType'][0], Mask):
            register_df_copy.at[i, 'maskType'] = [x.uid.render_pointer() for x in register_df_copy.at[i, 'maskType']]

bitMasks_df_copy = bitMasks_df.copy()
for i in bitMasks_df_copy.index:
    bitMasks_df_copy.at[i, 'name'] = bitMasks_df_copy.at[i, 'uid'].render_pointer()

groupMasks_df_copy = groupMasks_df.copy()
for i in groupMasks_df_copy.index:
    groupMasks_df_copy.at[i, 'name'] = groupMasks_df_copy.at[i, 'uid'].render_pointer()


AttributeError: 'NoneType' object has no attribute 'copy'

In [17]:
txt = f"""# Registers

{register_df_copy.to_markdown()}


# Masks

## Bit Masks

{bitMasks_df_copy.to_markdown()}


# IOs

{pinMapping_df.to_markdown()}
"""

with open("documentation.md", "w") as text_file:
    text_file.write(txt)

import markdown
with open("documentation.html", "w") as text_file:
    text_file.write(markdown.markdown(txt))

NameError: name 'groupMasks_df_copy' is not defined